In [1]:
import os
import shutil
import zipfile
import random
import math
import re
import numpy as np
import cv2
from collections import defaultdict, Counter

In [2]:
from object_detection.utils import label_map_util
label_map = label_map_util.load_labelmap('../annotations/label_map.pbtxt')
label_map_dict = label_map_util.get_label_map_dict(label_map)
label_map_dict

{'car': 1, 'truck': 2, 'bus': 3, 'bike': 4, 'person': 5}

Convert the label files to our default XML format, and remove labels that we don't care about / remove images that do not contain any obj. of interest.

In [110]:
def addObject(label, coords, truncated=0, difficult=0):
    out = '''\t<object>
    \t<name>{}</name>
    \t<pose>Unspecified</pose>
    \t<truncated>{}</truncated>
    \t<difficult>{}</difficult>
    \t<bndbox>
    \t\t<xmin>{}</xmin>
    \t\t<ymin>{}</ymin>
    \t\t<xmax>{}</xmax>
    \t\t<ymax>{}</ymax>
    \t</bndbox>\n\t</object>\n'''.format(
        label,
        int(truncated),
        int(difficult),
        int(coords[0]),
        int(coords[1]),
        int(coords[2]),
        int(coords[3])
    )
    return out

def convertToXMLFormat(path, func, flags=cv2.IMREAD_COLOR):
    path = path.strip().lower()
    tmp = path.replace('\\', '/').split('/')
    file, folder = tmp[-1], tmp[-2]
    img = cv2.imread(path, flags)
    out = '''<annotation>
    <folder>{}</folder>
    <filename>{}</filename>
    <path>{}</path>
    <source>
    \t<database>Unknown</database>
    </source>
    <size>
    \t<width>{}</width>
    \t<height>{}</height>
    \t<depth>3</depth>
    </size>
    <segmented>0</segmented>\n{}</annotation>'''.format(
        folder,
        file,
        path,
        img.shape[1],
        img.shape[0],
        func(path)
    )
    img = None
    return out

In [111]:
def convertFromXDark(path):
    out = ''
    with open(path.replace('.png', '.xml').replace('.jpg', '.xml').replace('.jpeg', '.xml'), 'r') as f:
        for i, line in enumerate(f):
            if i == 0: # Skip Top line.
                continue
                
            lbl, xmin, ymin, xmax, ymax = line.strip().lower().split()[:5]
            if lbl == 'motorbike':
                lbl = 'bike'
            elif lbl == 'people':
                lbl = 'person'

            if (not lbl in label_map_dict):
                continue
            
            out += addObject(lbl, [int(xmin), int(ymin), int(xmin)+int(xmax), int(ymin)+int(ymax)])
    return out

In [112]:
def convertFromKITTI(path):
    out = ''
    with open(path.replace('.png', '.xml').replace('.jpg', '.xml').replace('.jpeg', '.xml'), 'r') as f:
        for line in f:
            lbl, trunc, occlus, alpha, xmin, ymin, xmax, ymax = line.strip().lower().split()[:8]
            if lbl == 'van':
                lbl = 'car'
            elif lbl == 'pedestrian':
                lbl = 'person'

            if (not lbl in label_map_dict):
                continue
            
            out += addObject(lbl, [float(xmin), float(ymin), float(xmax), float(ymax)], (float(trunc) > 0.5), (int(occlus) >= 2))
    return out

Rename files of interest

In [16]:
def rename(path, ext):
    for f in os.listdir(path):
        f = f.strip().lower()
        os.rename('{}/{}'.format(path, f), '{}/{}.{}'.format(path, f.split('.')[0], ext))
        print(f, '--->', '{}.{}'.format(f.split('.')[0], ext))

In [17]:
rename('../images/temp', 'xml')

2015_01879.jpg.txt ---> 2015_01879.xml
2015_01880.png.txt ---> 2015_01880.xml
2015_01881.png.txt ---> 2015_01881.xml
2015_01882.png.txt ---> 2015_01882.xml
2015_01883.jpg.txt ---> 2015_01883.xml
2015_01884.png.txt ---> 2015_01884.xml
2015_01885.png.txt ---> 2015_01885.xml
2015_01886.png.txt ---> 2015_01886.xml
2015_01887.png.txt ---> 2015_01887.xml
2015_01888.jpg.txt ---> 2015_01888.xml
2015_01889.jpg.txt ---> 2015_01889.xml
2015_01890.png.txt ---> 2015_01890.xml
2015_01891.jpg.txt ---> 2015_01891.xml
2015_01892.jpg.txt ---> 2015_01892.xml
2015_01893.jpg.txt ---> 2015_01893.xml
2015_01894.jpg.txt ---> 2015_01894.xml
2015_01895.jpg.txt ---> 2015_01895.xml
2015_01896.jpg.txt ---> 2015_01896.xml
2015_01897.jpg.txt ---> 2015_01897.xml
2015_01898.jpg.txt ---> 2015_01898.xml
2015_01899.jpg.txt ---> 2015_01899.xml
2015_01900.jpg.txt ---> 2015_01900.xml
2015_01901.jpg.txt ---> 2015_01901.xml
2015_01902.jpg.txt ---> 2015_01902.xml
2015_01903.jpg.txt ---> 2015_01903.xml
2015_01904.jpg.txt ---> 2

2015_02235.jpg.txt ---> 2015_02235.xml
2015_02236.jpg.txt ---> 2015_02236.xml
2015_02237.jpeg.txt ---> 2015_02237.xml
2015_02238.jpg.txt ---> 2015_02238.xml
2015_02239.jpeg.txt ---> 2015_02239.xml
2015_02240.jpeg.txt ---> 2015_02240.xml
2015_02241.jpeg.txt ---> 2015_02241.xml
2015_02242.jpg.txt ---> 2015_02242.xml
2015_02243.jpg.txt ---> 2015_02243.xml
2015_02244.jpg.txt ---> 2015_02244.xml
2015_02245.jpg.txt ---> 2015_02245.xml
2015_02246.jpg.txt ---> 2015_02246.xml
2015_02247.jpg.txt ---> 2015_02247.xml
2015_02248.jpg.txt ---> 2015_02248.xml
2015_02249.jpg.txt ---> 2015_02249.xml
2015_02250.jpg.txt ---> 2015_02250.xml
2015_02251.jpg.txt ---> 2015_02251.xml
2015_02252.jpg.txt ---> 2015_02252.xml
2015_02253.jpg.txt ---> 2015_02253.xml
2015_02254.jpg.txt ---> 2015_02254.xml
2015_02255.jpg.txt ---> 2015_02255.xml
2015_02256.jpg.txt ---> 2015_02256.xml
2015_02257.jpg.txt ---> 2015_02257.xml
2015_02258.jpg.txt ---> 2015_02258.xml
2015_02259.jpg.txt ---> 2015_02259.xml
2015_02260.jpg.txt --

2015_02586.jpg.txt ---> 2015_02586.xml
2015_02587.jpg.txt ---> 2015_02587.xml
2015_02588.jpg.txt ---> 2015_02588.xml
2015_02589.jpg.txt ---> 2015_02589.xml
2015_02590.jpg.txt ---> 2015_02590.xml
2015_02591.jpg.txt ---> 2015_02591.xml
2015_02592.jpg.txt ---> 2015_02592.xml
2015_02593.jpg.txt ---> 2015_02593.xml
2015_02594.jpg.txt ---> 2015_02594.xml
2015_02595.jpg.txt ---> 2015_02595.xml
2015_02596.jpg.txt ---> 2015_02596.xml
2015_02597.jpg.txt ---> 2015_02597.xml
2015_02598.jpg.txt ---> 2015_02598.xml
2015_02599.jpg.txt ---> 2015_02599.xml
2015_02600.jpg.txt ---> 2015_02600.xml
2015_02601.jpg.txt ---> 2015_02601.xml
2015_02602.jpg.txt ---> 2015_02602.xml
2015_02603.jpg.txt ---> 2015_02603.xml
2015_02604.jpg.txt ---> 2015_02604.xml
2015_02605.jpg.txt ---> 2015_02605.xml
2015_02606.jpg.txt ---> 2015_02606.xml
2015_02607.jpg.txt ---> 2015_02607.xml
2015_02608.jpg.txt ---> 2015_02608.xml
2015_02609.jpg.txt ---> 2015_02609.xml
2015_02610.jpg.txt ---> 2015_02610.xml
2015_02611.jpg.txt ---> 2

2015_02940.jpg.txt ---> 2015_02940.xml
2015_02941.jpg.txt ---> 2015_02941.xml
2015_02942.jpg.txt ---> 2015_02942.xml
2015_02943.jpg.txt ---> 2015_02943.xml
2015_02944.jpg.txt ---> 2015_02944.xml
2015_02945.jpg.txt ---> 2015_02945.xml
2015_02946.jpg.txt ---> 2015_02946.xml
2015_02947.jpg.txt ---> 2015_02947.xml
2015_02948.jpg.txt ---> 2015_02948.xml
2015_02949.jpg.txt ---> 2015_02949.xml
2015_02950.jpg.txt ---> 2015_02950.xml
2015_02951.jpg.txt ---> 2015_02951.xml
2015_02952.jpg.txt ---> 2015_02952.xml
2015_02953.jpg.txt ---> 2015_02953.xml
2015_02954.jpg.txt ---> 2015_02954.xml
2015_02955.jpg.txt ---> 2015_02955.xml
2015_02956.png.txt ---> 2015_02956.xml
2015_02957.jpg.txt ---> 2015_02957.xml
2015_02958.jpg.txt ---> 2015_02958.xml
2015_02959.jpg.txt ---> 2015_02959.xml
2015_02960.jpg.txt ---> 2015_02960.xml
2015_02961.jpg.txt ---> 2015_02961.xml
2015_02962.jpg.txt ---> 2015_02962.xml
2015_02963.png.txt ---> 2015_02963.xml
2015_02964.jpg.txt ---> 2015_02964.xml
2015_02965.jpg.txt ---> 2

2015_05987.jpg.txt ---> 2015_05987.xml
2015_05988.jpg.txt ---> 2015_05988.xml
2015_05989.jpg.txt ---> 2015_05989.xml
2015_05990.jpg.txt ---> 2015_05990.xml
2015_05991.jpg.txt ---> 2015_05991.xml
2015_05992.jpg.txt ---> 2015_05992.xml
2015_05993.jpg.txt ---> 2015_05993.xml
2015_05994.jpg.txt ---> 2015_05994.xml
2015_05995.jpg.txt ---> 2015_05995.xml
2015_05996.jpg.txt ---> 2015_05996.xml
2015_05997.jpg.txt ---> 2015_05997.xml
2015_05998.jpg.txt ---> 2015_05998.xml
2015_05999.jpg.txt ---> 2015_05999.xml
2015_06000.jpg.txt ---> 2015_06000.xml
2015_06001.jpg.txt ---> 2015_06001.xml
2015_06002.jpg.txt ---> 2015_06002.xml
2015_06003.jpg.txt ---> 2015_06003.xml
2015_06004.jpg.txt ---> 2015_06004.xml
2015_06005.jpg.txt ---> 2015_06005.xml
2015_06006.jpg.txt ---> 2015_06006.xml
2015_06007.jpg.txt ---> 2015_06007.xml
2015_06008.jpg.txt ---> 2015_06008.xml
2015_06009.jpg.txt ---> 2015_06009.xml
2015_06010.jpg.txt ---> 2015_06010.xml
2015_06011.jpg.txt ---> 2015_06011.xml
2015_06012.jpg.txt ---> 2

2015_06243.jpg.txt ---> 2015_06243.xml
2015_06244.jpg.txt ---> 2015_06244.xml
2015_06245.jpg.txt ---> 2015_06245.xml
2015_06246.jpg.txt ---> 2015_06246.xml
2015_06247.jpg.txt ---> 2015_06247.xml
2015_06248.jpg.txt ---> 2015_06248.xml
2015_06249.jpg.txt ---> 2015_06249.xml
2015_06250.jpg.txt ---> 2015_06250.xml
2015_06251.jpg.txt ---> 2015_06251.xml
2015_06252.jpg.txt ---> 2015_06252.xml
2015_06253.jpg.txt ---> 2015_06253.xml
2015_06254.jpg.txt ---> 2015_06254.xml
2015_06255.jpg.txt ---> 2015_06255.xml
2015_06256.jpg.txt ---> 2015_06256.xml
2015_06257.jpg.txt ---> 2015_06257.xml
2015_06258.jpg.txt ---> 2015_06258.xml
2015_06259.jpg.txt ---> 2015_06259.xml
2015_06260.jpg.txt ---> 2015_06260.xml
2015_06261.jpg.txt ---> 2015_06261.xml
2015_06262.jpg.txt ---> 2015_06262.xml
2015_06263.jpg.txt ---> 2015_06263.xml
2015_06264.jpg.txt ---> 2015_06264.xml
2015_06265.jpg.txt ---> 2015_06265.xml
2015_06266.jpg.txt ---> 2015_06266.xml
2015_06267.jpg.txt ---> 2015_06267.xml
2015_06268.jpg.txt ---> 2

2015_06577.jpg.txt ---> 2015_06577.xml
2015_06578.jpg.txt ---> 2015_06578.xml
2015_06579.jpg.txt ---> 2015_06579.xml
2015_06580.jpg.txt ---> 2015_06580.xml
2015_06581.jpg.txt ---> 2015_06581.xml
2015_06582.jpg.txt ---> 2015_06582.xml
2015_06583.jpg.txt ---> 2015_06583.xml
2015_06584.jpg.txt ---> 2015_06584.xml
2015_06585.jpg.txt ---> 2015_06585.xml
2015_06586.jpg.txt ---> 2015_06586.xml
2015_06587.jpg.txt ---> 2015_06587.xml
2015_06588.jpg.txt ---> 2015_06588.xml
2015_06589.jpg.txt ---> 2015_06589.xml
2015_06590.jpg.txt ---> 2015_06590.xml
2015_06591.jpg.txt ---> 2015_06591.xml
2015_06592.jpg.txt ---> 2015_06592.xml
2015_06593.jpg.txt ---> 2015_06593.xml
2015_06594.jpg.txt ---> 2015_06594.xml
2015_06595.jpg.txt ---> 2015_06595.xml
2015_06596.jpg.txt ---> 2015_06596.xml
2015_06597.jpg.txt ---> 2015_06597.xml
2015_06598.jpg.txt ---> 2015_06598.xml
2015_06599.jpg.txt ---> 2015_06599.xml
2015_06600.jpg.txt ---> 2015_06600.xml
2015_06601.jpg.txt ---> 2015_06601.xml
2015_06602.jpg.txt ---> 2

In [11]:
rename('../images/temp', 'jpg')

2015_01879.jpg ---> 2015_01879.jpg
2015_01880.jpg ---> 2015_01880.jpg
2015_01881.jpg ---> 2015_01881.jpg
2015_01882.jpg ---> 2015_01882.jpg
2015_01883.jpg ---> 2015_01883.jpg
2015_01884.jpg ---> 2015_01884.jpg
2015_01885.jpg ---> 2015_01885.jpg
2015_01886.jpg ---> 2015_01886.jpg
2015_01887.jpg ---> 2015_01887.jpg
2015_01888.jpg ---> 2015_01888.jpg
2015_01889.jpg ---> 2015_01889.jpg
2015_01890.jpg ---> 2015_01890.jpg
2015_01891.jpg ---> 2015_01891.jpg
2015_01892.jpg ---> 2015_01892.jpg
2015_01893.jpg ---> 2015_01893.jpg
2015_01894.jpg ---> 2015_01894.jpg
2015_01895.jpg ---> 2015_01895.jpg
2015_01896.jpg ---> 2015_01896.jpg
2015_01897.jpg ---> 2015_01897.jpg
2015_01898.jpg ---> 2015_01898.jpg
2015_01899.jpg ---> 2015_01899.jpg
2015_01900.jpg ---> 2015_01900.jpg
2015_01901.jpg ---> 2015_01901.jpg
2015_01902.jpg ---> 2015_01902.jpg
2015_01903.jpg ---> 2015_01903.jpg
2015_01904.jpg ---> 2015_01904.jpg
2015_01905.jpg ---> 2015_01905.jpg
2015_01906.jpg ---> 2015_01906.jpg
2015_01907.jpg ---> 

2015_02277.jpg ---> 2015_02277.jpg
2015_02278.jpg ---> 2015_02278.jpg
2015_02279.jpg ---> 2015_02279.jpg
2015_02280.jpg ---> 2015_02280.jpg
2015_02281.jpg ---> 2015_02281.jpg
2015_02282.jpg ---> 2015_02282.jpg
2015_02283.jpg ---> 2015_02283.jpg
2015_02284.jpg ---> 2015_02284.jpg
2015_02285.jpg ---> 2015_02285.jpg
2015_02286.jpg ---> 2015_02286.jpg
2015_02287.jpg ---> 2015_02287.jpg
2015_02288.jpg ---> 2015_02288.jpg
2015_02289.jpg ---> 2015_02289.jpg
2015_02290.jpg ---> 2015_02290.jpg
2015_02291.jpg ---> 2015_02291.jpg
2015_02292.jpg ---> 2015_02292.jpg
2015_02293.jpg ---> 2015_02293.jpg
2015_02294.jpg ---> 2015_02294.jpg
2015_02295.jpg ---> 2015_02295.jpg
2015_02296.jpg ---> 2015_02296.jpg
2015_02297.jpg ---> 2015_02297.jpg
2015_02298.jpg ---> 2015_02298.jpg
2015_02299.jpg ---> 2015_02299.jpg
2015_02300.jpg ---> 2015_02300.jpg
2015_02301.jpg ---> 2015_02301.jpg
2015_02302.jpg ---> 2015_02302.jpg
2015_02303.jpg ---> 2015_02303.jpg
2015_02304.jpg ---> 2015_02304.jpg
2015_02305.jpg ---> 

2015_02666.jpg ---> 2015_02666.jpg
2015_02667.jpg ---> 2015_02667.jpg
2015_02668.jpg ---> 2015_02668.jpg
2015_02669.jpg ---> 2015_02669.jpg
2015_02670.jpg ---> 2015_02670.jpg
2015_02671.jpg ---> 2015_02671.jpg
2015_02672.jpg ---> 2015_02672.jpg
2015_02673.jpg ---> 2015_02673.jpg
2015_02674.jpg ---> 2015_02674.jpg
2015_02675.jpg ---> 2015_02675.jpg
2015_02676.jpg ---> 2015_02676.jpg
2015_02677.jpg ---> 2015_02677.jpg
2015_02678.jpg ---> 2015_02678.jpg
2015_02679.jpg ---> 2015_02679.jpg
2015_02680.jpg ---> 2015_02680.jpg
2015_02681.jpg ---> 2015_02681.jpg
2015_02682.jpg ---> 2015_02682.jpg
2015_02683.jpg ---> 2015_02683.jpg
2015_02684.jpg ---> 2015_02684.jpg
2015_02685.jpg ---> 2015_02685.jpg
2015_02686.jpg ---> 2015_02686.jpg
2015_02687.jpg ---> 2015_02687.jpg
2015_02688.jpg ---> 2015_02688.jpg
2015_02689.jpg ---> 2015_02689.jpg
2015_02690.jpg ---> 2015_02690.jpg
2015_02691.jpg ---> 2015_02691.jpg
2015_02692.jpg ---> 2015_02692.jpg
2015_02693.jpg ---> 2015_02693.jpg
2015_02694.jpg ---> 

2015_05759.jpg ---> 2015_05759.jpg
2015_05760.jpg ---> 2015_05760.jpg
2015_05761.jpg ---> 2015_05761.jpg
2015_05762.jpg ---> 2015_05762.jpg
2015_05763.jpg ---> 2015_05763.jpg
2015_05764.jpg ---> 2015_05764.jpg
2015_05765.jpg ---> 2015_05765.jpg
2015_05766.jpg ---> 2015_05766.jpg
2015_05767.jpg ---> 2015_05767.jpg
2015_05768.jpg ---> 2015_05768.jpg
2015_05769.jpg ---> 2015_05769.jpg
2015_05770.jpg ---> 2015_05770.jpg
2015_05771.jpg ---> 2015_05771.jpg
2015_05772.jpg ---> 2015_05772.jpg
2015_05773.jpg ---> 2015_05773.jpg
2015_05774.jpg ---> 2015_05774.jpg
2015_05775.jpg ---> 2015_05775.jpg
2015_05776.jpg ---> 2015_05776.jpg
2015_05777.jpg ---> 2015_05777.jpg
2015_05778.jpg ---> 2015_05778.jpg
2015_05779.jpg ---> 2015_05779.jpg
2015_05780.jpg ---> 2015_05780.jpg
2015_05781.jpg ---> 2015_05781.jpg
2015_05782.jpg ---> 2015_05782.jpg
2015_05783.jpg ---> 2015_05783.jpg
2015_05784.jpg ---> 2015_05784.jpg
2015_05785.jpg ---> 2015_05785.jpg
2015_05786.jpg ---> 2015_05786.jpg
2015_05787.jpg ---> 

2015_06147.jpeg ---> 2015_06147.jpg
2015_06148.jpeg ---> 2015_06148.jpg
2015_06149.jpeg ---> 2015_06149.jpg
2015_06150.jpeg ---> 2015_06150.jpg
2015_06151.jpeg ---> 2015_06151.jpg
2015_06152.jpeg ---> 2015_06152.jpg
2015_06153.jpeg ---> 2015_06153.jpg
2015_06154.jpeg ---> 2015_06154.jpg
2015_06155.jpeg ---> 2015_06155.jpg
2015_06156.jpeg ---> 2015_06156.jpg
2015_06157.jpg ---> 2015_06157.jpg
2015_06158.jpg ---> 2015_06158.jpg
2015_06159.jpg ---> 2015_06159.jpg
2015_06160.jpg ---> 2015_06160.jpg
2015_06161.jpg ---> 2015_06161.jpg
2015_06162.jpg ---> 2015_06162.jpg
2015_06163.jpg ---> 2015_06163.jpg
2015_06164.jpg ---> 2015_06164.jpg
2015_06165.jpg ---> 2015_06165.jpg
2015_06166.jpg ---> 2015_06166.jpg
2015_06167.jpg ---> 2015_06167.jpg
2015_06168.jpg ---> 2015_06168.jpg
2015_06169.jpg ---> 2015_06169.jpg
2015_06170.jpg ---> 2015_06170.jpg
2015_06171.jpg ---> 2015_06171.jpg
2015_06172.jpg ---> 2015_06172.jpg
2015_06173.jpg ---> 2015_06173.jpg
2015_06174.jpg ---> 2015_06174.jpg
2015_06175

2015_06547.jpg ---> 2015_06547.jpg
2015_06548.jpg ---> 2015_06548.jpg
2015_06549.jpg ---> 2015_06549.jpg
2015_06550.jpg ---> 2015_06550.jpg
2015_06551.jpg ---> 2015_06551.jpg
2015_06552.jpg ---> 2015_06552.jpg
2015_06553.jpg ---> 2015_06553.jpg
2015_06554.jpg ---> 2015_06554.jpg
2015_06555.jpg ---> 2015_06555.jpg
2015_06556.jpg ---> 2015_06556.jpg
2015_06557.jpg ---> 2015_06557.jpg
2015_06558.jpg ---> 2015_06558.jpg
2015_06559.jpg ---> 2015_06559.jpg
2015_06560.jpg ---> 2015_06560.jpg
2015_06561.jpg ---> 2015_06561.jpg
2015_06562.jpg ---> 2015_06562.jpg
2015_06563.jpg ---> 2015_06563.jpg
2015_06564.jpg ---> 2015_06564.jpg
2015_06565.jpg ---> 2015_06565.jpg
2015_06566.jpg ---> 2015_06566.jpg
2015_06567.jpg ---> 2015_06567.jpg
2015_06568.jpg ---> 2015_06568.jpg
2015_06569.jpg ---> 2015_06569.jpg
2015_06570.jpg ---> 2015_06570.jpg
2015_06571.jpg ---> 2015_06571.jpg
2015_06572.jpg ---> 2015_06572.jpg
2015_06573.jpg ---> 2015_06573.jpg
2015_06574.jpg ---> 2015_06574.jpg
2015_06575.jpg ---> 

Convert png to jpg

In [10]:
for f in os.listdir('../images/temp'):
    f = f.strip().lower()    
    if f.endswith('.png'):
        print("Converting", f, 'to jpg!')
        img = cv2.imread('../images/temp/{}'.format(f), cv2.IMREAD_COLOR)   
        cv2.imwrite('../images/temp/{}.jpg'.format(f[:-4]), img, [int(cv2.IMWRITE_JPEG_QUALITY), 90])

Converting 2015_01880.png to jpg!
Converting 2015_01881.png to jpg!
Converting 2015_01882.png to jpg!
Converting 2015_01884.png to jpg!
Converting 2015_01885.png to jpg!
Converting 2015_01886.png to jpg!
Converting 2015_01887.png to jpg!
Converting 2015_01890.png to jpg!
Converting 2015_02309.png to jpg!
Converting 2015_02399.png to jpg!
Converting 2015_02400.png to jpg!
Converting 2015_02401.png to jpg!
Converting 2015_02402.png to jpg!
Converting 2015_02403.png to jpg!
Converting 2015_02410.png to jpg!
Converting 2015_02416.png to jpg!
Converting 2015_02705.png to jpg!
Converting 2015_02725.png to jpg!
Converting 2015_02761.png to jpg!
Converting 2015_02768.png to jpg!
Converting 2015_02810.png to jpg!
Converting 2015_02853.png to jpg!
Converting 2015_02864.png to jpg!
Converting 2015_02902.png to jpg!
Converting 2015_02903.png to jpg!
Converting 2015_02956.png to jpg!
Converting 2015_02963.png to jpg!
Converting 2015_03004.png to jpg!
Converting 2015_03005.png to jpg!
Converting 201

Read the contents and write it out as xml format, with label changes etc.

In [12]:
labels = defaultdict(int)
for f in set([file[:-4] for file in os.listdir('../images/dark')]):
    with open('../images/dark/{}.xml'.format(f), 'r') as file:
        for i, line in enumerate(file):
            if i == 0:
                continue
            l = line.strip().lower().split()[0]
            if l == 'motorbike':
                l = 'bike'
            elif l == 'people':
                l = 'person'
                
            if (not l in label_map_dict):
                continue
            
            labels[l] += 1

In [34]:
labels

defaultdict(int, {'car': 2360, 'person': 4074, 'bike': 1000, 'bus': 685})

In [28]:
for f in set([file[:-4].lower() for file in os.listdir('../images/dark')]):
    out = convertToXMLFormat(r'E:\Cloud\GIT\CVEET\images\dark\{}.jpg'.format(f), convertFromXDark)
    with open(r'E:\Cloud\GIT\CVEET\images\dark\{}.xml'.format(f), 'w') as xml:
        xml.write('{}\n'.format(out))
        print('Processed', f)

Processed 2015_02480
Processed 2015_06047
Processed 2015_06054
Processed 2015_06719
Processed 2015_02582
Processed 2015_03025
Processed 2015_02888
Processed 2015_06479
Processed 2015_06349
Processed 2015_06781
Processed 2015_06360
Processed 2015_02321
Processed 2015_06039
Processed 2015_02468
Processed 2015_06301
Processed 2015_06197
Processed 2015_06646
Processed 2015_05826
Processed 2015_06098
Processed 2015_02260
Processed 2015_06185
Processed 2015_06372
Processed 2015_06721
Processed 2015_05965
Processed 2015_05982
Processed 2015_02907
Processed 2015_06362
Processed 2015_06224
Processed 2015_02799
Processed 2015_06157
Processed 2015_01928
Processed 2015_02789
Processed 2015_02411
Processed 2015_06306
Processed 2015_06573
Processed 2015_02910
Processed 2015_06167
Processed 2015_06645
Processed 2015_05900
Processed 2015_06644
Processed 2015_02890
Processed 2015_06065
Processed 2015_02130
Processed 2015_02865
Processed 2015_06278
Processed 2015_02589
Processed 2015_06510
Processed 201

Processed 2015_02143
Processed 2015_06706
Processed 2015_06788
Processed 2015_02571
Processed 2015_02280
Processed 2015_02594
Processed 2015_05765
Processed 2015_02179
Processed 2015_06186
Processed 2015_02529
Processed 2015_06160
Processed 2015_06264
Processed 2015_06158
Processed 2015_02882
Processed 2015_06109
Processed 2015_02715
Processed 2015_06240
Processed 2015_02638
Processed 2015_02356
Processed 2015_02903
Processed 2015_06741
Processed 2015_02583
Processed 2015_02913
Processed 2015_02317
Processed 2015_02307
Processed 2015_06071
Processed 2015_02692
Processed 2015_02820
Processed 2015_06363
Processed 2015_02863
Processed 2015_06021
Processed 2015_02553
Processed 2015_05818
Processed 2015_01938
Processed 2015_06497
Processed 2015_02312
Processed 2015_06261
Processed 2015_06810
Processed 2015_02403
Processed 2015_06087
Processed 2015_05771
Processed 2015_02076
Processed 2015_06481
Processed 2015_02967
Processed 2015_06436
Processed 2015_06142
Processed 2015_02651
Processed 201

Processed 2015_06615
Processed 2015_03010
Processed 2015_06104
Processed 2015_06281
Processed 2015_02821
Processed 2015_06003
Processed 2015_06647
Processed 2015_05968
Processed 2015_02296
Processed 2015_02596
Processed 2015_02934
Processed 2015_02630
Processed 2015_06356
Processed 2015_06713
Processed 2015_02346
Processed 2015_02347
Processed 2015_03017
Processed 2015_02084
Processed 2015_02643
Processed 2015_05817
Processed 2015_06082
Processed 2015_06090
Processed 2015_05805
Processed 2015_06439
Processed 2015_02176
Processed 2015_06202
Processed 2015_02222
Processed 2015_02498
Processed 2015_06599
Processed 2015_06640
Processed 2015_05830
Processed 2015_06174
Processed 2015_06309
Processed 2015_02993
Processed 2015_02431
Processed 2015_02561
Processed 2015_06233
Processed 2015_05810
Processed 2015_06693
Processed 2015_06444
Processed 2015_06568
Processed 2015_02499
Processed 2015_06668
Processed 2015_06771
Processed 2015_06790
Processed 2015_02409
Processed 2015_02186
Processed 201

Processed 2015_06046
Processed 2015_02602
Processed 2015_06574
Processed 2015_06323
Processed 2015_06040
Processed 2015_05959
Processed 2015_05879
Processed 2015_02734
Processed 2015_06263
Processed 2015_02099
Processed 2015_06669
Processed 2015_02850
Processed 2015_02239
Processed 2015_02568
Processed 2015_05945
Processed 2015_02898
Processed 2015_02640
Processed 2015_02194
Processed 2015_05807
Processed 2015_02941
Processed 2015_06551
Processed 2015_06350
Processed 2015_02991
Processed 2015_05791
Processed 2015_02116
Processed 2015_05784
Processed 2015_02649
Processed 2015_06431
Processed 2015_02737
Processed 2015_06486
Processed 2015_06106
Processed 2015_06827
Processed 2015_06236
Processed 2015_02085
Processed 2015_02011
Processed 2015_02629
Processed 2015_05763
Processed 2015_01921
Processed 2015_05752
Processed 2015_01981
Processed 2015_05829
Processed 2015_02265
Processed 2015_01984
Processed 2015_05790
Processed 2015_02710
Processed 2015_02519
Processed 2015_02895
Processed 201

Processed 2015_02827
Processed 2015_02196
Processed 2015_01957
Processed 2015_06149
Processed 2015_02515
Processed 2015_02626
Processed 2015_05988
Processed 2015_06498
Processed 2015_02461
Processed 2015_02755
Processed 2015_02614
Processed 2015_06624
Processed 2015_01998
Processed 2015_02198
Processed 2015_06187
Processed 2015_06126
Processed 2015_06400
Processed 2015_02728
Processed 2015_03004
Processed 2015_06694
Processed 2015_06749
Processed 2015_06248
Processed 2015_06531
Processed 2015_02593
Processed 2015_06505
Processed 2015_02696
Processed 2015_06621
Processed 2015_02949
Processed 2015_05884
Processed 2015_06251
Processed 2015_05834
Processed 2015_06501
Processed 2015_02686
Processed 2015_02237
Processed 2015_02183
Processed 2015_02810
Processed 2015_03009
Processed 2015_05748
Processed 2015_06418
Processed 2015_02333
Processed 2015_06564
Processed 2015_02046
Processed 2015_06086
Processed 2015_05917
Processed 2015_02576
Processed 2015_02578
Processed 2015_02684
Processed 201

Processed 2015_06359
Processed 2015_02702
Processed 2015_02791
Processed 2015_06155
Processed 2015_06025
Processed 2015_02932
Processed 2015_02086
Processed 2015_06027
Processed 2015_06785
Processed 2015_01948
Processed 2015_02310
Processed 2015_06150
Processed 2015_06080
Processed 2015_02822
Processed 2015_05957
Processed 2015_05986
Processed 2015_05994
Processed 2015_02305
Processed 2015_02322
Processed 2015_05907
Processed 2015_06515
Processed 2015_02249
Processed 2015_02747
Processed 2015_06322
Processed 2015_06422
Processed 2015_06642
Processed 2015_02160
Processed 2015_02012
Processed 2015_02278
Processed 2015_06824
Processed 2015_02414
Processed 2015_05836
Processed 2015_06557
Processed 2015_02985
Processed 2015_06596
Processed 2015_02720
Processed 2015_02170
Processed 2015_02349
Processed 2015_02534
Processed 2015_02974
Processed 2015_02361
Processed 2015_06262
Processed 2015_06606
Processed 2015_06097
Processed 2015_02704
Processed 2015_02402
Processed 2015_02563
Processed 201

Converting the KITTI set

In [43]:
def stat(files):
    labels = defaultdict(int)
    for f in files:
        with open('../images/temp/{}'.format(f), 'r') as tmp:
            for txt in tmp:
                txt = txt.strip().lower().split()[0]
                #if txt == 'pedestrian':
                #    txt = 'person'
                #elif txt == 'van':
                #txt = 'car'

                #if (not txt in label_map_dict):
                    #continue

                labels[txt] += 1
    return labels

In [12]:
labelFiles = [f.lower() for f in os.listdir('../images/temp')]
random.shuffle(labelFiles)
random.shuffle(labelFiles)
#labelFiles = labelFiles[:int(len(labelFiles)*0.35)]
#labelFiles

In [44]:
len(labelFiles)

7481

In [11]:
stat(labelFiles)

defaultdict(int,
            {'truck': 1094,
             'car': 28742,
             'dontcare': 11295,
             'pedestrian': 4487,
             'cyclist': 1627,
             'van': 2914,
             'misc': 973,
             'tram': 511,
             'person_sitting': 222})

This dataset is very large, so we try to find a decent amount of each label.

In [45]:
filesOfInterest = []
labels = defaultdict(int)
for f in labelFiles:
    if (labels['car'] > 1000) and (labels['van'] > 500) and (labels['truck'] > 600) and (labels['person'] > 1250):
        print("DONE")
        break
    filesOfInterest.append(f)
    with open('../images/temp/{}'.format(f), 'r') as tmp:
        for txt in tmp:
            txt = txt.strip().lower().split()[0]
            if txt == 'pedestrian':
                txt = 'person'
            labels[txt] += 1
print(labels)

DONE
defaultdict(<class 'int'>, {'car': 15113, 'dontcare': 5862, 'van': 1508, 'cyclist': 888, 'person': 2279, 'truck': 601, 'misc': 470, 'person_sitting': 111, 'tram': 266})


In [46]:
filesOfInterest[:50]

['007438.txt',
 '005618.txt',
 '002239.txt',
 '001362.txt',
 '003385.txt',
 '001351.txt',
 '002505.txt',
 '003705.txt',
 '005860.txt',
 '007475.txt',
 '003386.txt',
 '001502.txt',
 '002079.txt',
 '000929.txt',
 '007356.txt',
 '004592.txt',
 '001815.txt',
 '000960.txt',
 '005582.txt',
 '004284.txt',
 '006543.txt',
 '006599.txt',
 '003681.txt',
 '007282.txt',
 '004023.txt',
 '004513.txt',
 '000455.txt',
 '001049.txt',
 '007017.txt',
 '006767.txt',
 '003863.txt',
 '002459.txt',
 '006457.txt',
 '003197.txt',
 '001724.txt',
 '004381.txt',
 '003205.txt',
 '001485.txt',
 '004429.txt',
 '003337.txt',
 '004033.txt',
 '004163.txt',
 '002493.txt',
 '000110.txt',
 '005732.txt',
 '000846.txt',
 '003791.txt',
 '005879.txt',
 '002557.txt',
 '000403.txt']

In [47]:
len(filesOfInterest)

3970

In [48]:
file_stat = []
for f in filesOfInterest:
    lbls = defaultdict(int)
    with open('../images/temp/{}'.format(f), 'r') as tmp:
        for txt in tmp:
            txt = txt.strip().lower().split()[0]
            if txt == 'pedestrian':
                txt = 'person'
            lbls[txt] += 1
            
    if lbls['car'] > 4:
        continue
            
    file_stat.append((f, lbls))

In [49]:
file_stat.sort(key=lambda x: x[1]['car'], reverse=True)

In [50]:
file_stat

[('002505.txt', defaultdict(int, {'person': 1, 'car': 4, 'truck': 1})),
 ('004592.txt', defaultdict(int, {'car': 4})),
 ('001815.txt', defaultdict(int, {'truck': 1, 'car': 4, 'dontcare': 1})),
 ('004023.txt', defaultdict(int, {'van': 1, 'car': 4, 'dontcare': 2})),
 ('001049.txt', defaultdict(int, {'car': 4, 'person': 10, 'dontcare': 2})),
 ('007041.txt',
  defaultdict(int, {'car': 4, 'truck': 1, 'van': 1, 'dontcare': 2})),
 ('001668.txt', defaultdict(int, {'car': 4, 'dontcare': 1})),
 ('006571.txt',
  defaultdict(int, {'car': 4, 'van': 1, 'truck': 1, 'dontcare': 1})),
 ('000739.txt', defaultdict(int, {'car': 4})),
 ('003127.txt', defaultdict(int, {'car': 4, 'dontcare': 1})),
 ('005239.txt',
  defaultdict(int, {'car': 4, 'van': 1, 'person': 3, 'dontcare': 2})),
 ('003244.txt', defaultdict(int, {'car': 4, 'cyclist': 1, 'dontcare': 4})),
 ('000804.txt',
  defaultdict(int, {'car': 4, 'cyclist': 1, 'person': 1, 'dontcare': 1})),
 ('001499.txt', defaultdict(int, {'car': 4, 'dontcare': 2})),


In [51]:
stat([f for f,_ in file_stat])

defaultdict(int,
            {'pedestrian': 1956,
             'car': 4899,
             'truck': 412,
             'dontcare': 3907,
             'van': 724,
             'cyclist': 668,
             'misc': 247,
             'tram': 240,
             'person_sitting': 105})

In [58]:
for f in [f for f,_ in file_stat]:
    shutil.move('../images/temp/{}'.format(f), '../images/kitti/{}.xml'.format(f.lower()[:-4]))

In [82]:
with zipfile.ZipFile(r'E:\TEMP\KITTI-DATA.zip') as z:
    for file in sorted(list(set([f.lower()[:-4] for f,_ in file_stat]))):
        with z.open('kitti_single/training/image_2/{}.png'.format(file)) as archFile:
            img = cv2.imdecode(np.frombuffer(archFile.read(), dtype=np.uint8), cv2.IMREAD_COLOR)
            cv2.imwrite('../images/temp/{}.jpg'.format(file), img, [int(cv2.IMWRITE_JPEG_QUALITY), 90])
            print('Extracted:', file)
        #break

Extracted: 000000
Extracted: 000001
Extracted: 000003
Extracted: 000004
Extracted: 000007
Extracted: 000011
Extracted: 000015
Extracted: 000016
Extracted: 000020
Extracted: 000023
Extracted: 000026
Extracted: 000027
Extracted: 000028
Extracted: 000033
Extracted: 000034
Extracted: 000037
Extracted: 000041
Extracted: 000050
Extracted: 000054
Extracted: 000055
Extracted: 000056
Extracted: 000057
Extracted: 000058
Extracted: 000059
Extracted: 000060
Extracted: 000062
Extracted: 000063
Extracted: 000064
Extracted: 000072
Extracted: 000073
Extracted: 000078
Extracted: 000081
Extracted: 000084
Extracted: 000087
Extracted: 000088
Extracted: 000101
Extracted: 000102
Extracted: 000103
Extracted: 000106
Extracted: 000108
Extracted: 000115
Extracted: 000119
Extracted: 000126
Extracted: 000132
Extracted: 000133
Extracted: 000134
Extracted: 000140
Extracted: 000147
Extracted: 000148
Extracted: 000151
Extracted: 000155
Extracted: 000158
Extracted: 000160
Extracted: 000162
Extracted: 000164
Extracted:

Extracted: 001291
Extracted: 001302
Extracted: 001303
Extracted: 001307
Extracted: 001314
Extracted: 001315
Extracted: 001317
Extracted: 001320
Extracted: 001324
Extracted: 001326
Extracted: 001328
Extracted: 001329
Extracted: 001331
Extracted: 001333
Extracted: 001339
Extracted: 001341
Extracted: 001342
Extracted: 001343
Extracted: 001344
Extracted: 001345
Extracted: 001346
Extracted: 001347
Extracted: 001348
Extracted: 001350
Extracted: 001352
Extracted: 001356
Extracted: 001357
Extracted: 001358
Extracted: 001362
Extracted: 001365
Extracted: 001366
Extracted: 001372
Extracted: 001373
Extracted: 001377
Extracted: 001380
Extracted: 001381
Extracted: 001382
Extracted: 001385
Extracted: 001386
Extracted: 001387
Extracted: 001390
Extracted: 001398
Extracted: 001400
Extracted: 001411
Extracted: 001418
Extracted: 001419
Extracted: 001427
Extracted: 001430
Extracted: 001433
Extracted: 001436
Extracted: 001439
Extracted: 001440
Extracted: 001442
Extracted: 001448
Extracted: 001449
Extracted:

Extracted: 002614
Extracted: 002617
Extracted: 002618
Extracted: 002620
Extracted: 002628
Extracted: 002630
Extracted: 002633
Extracted: 002634
Extracted: 002642
Extracted: 002645
Extracted: 002646
Extracted: 002648
Extracted: 002649
Extracted: 002654
Extracted: 002655
Extracted: 002657
Extracted: 002660
Extracted: 002663
Extracted: 002664
Extracted: 002668
Extracted: 002669
Extracted: 002676
Extracted: 002679
Extracted: 002686
Extracted: 002689
Extracted: 002690
Extracted: 002693
Extracted: 002696
Extracted: 002699
Extracted: 002700
Extracted: 002701
Extracted: 002702
Extracted: 002703
Extracted: 002706
Extracted: 002709
Extracted: 002713
Extracted: 002718
Extracted: 002720
Extracted: 002722
Extracted: 002723
Extracted: 002724
Extracted: 002725
Extracted: 002728
Extracted: 002732
Extracted: 002735
Extracted: 002738
Extracted: 002739
Extracted: 002741
Extracted: 002742
Extracted: 002744
Extracted: 002745
Extracted: 002755
Extracted: 002758
Extracted: 002759
Extracted: 002760
Extracted:

Extracted: 003886
Extracted: 003892
Extracted: 003895
Extracted: 003897
Extracted: 003899
Extracted: 003905
Extracted: 003906
Extracted: 003907
Extracted: 003908
Extracted: 003909
Extracted: 003913
Extracted: 003914
Extracted: 003918
Extracted: 003921
Extracted: 003925
Extracted: 003926
Extracted: 003928
Extracted: 003935
Extracted: 003936
Extracted: 003939
Extracted: 003940
Extracted: 003946
Extracted: 003947
Extracted: 003952
Extracted: 003953
Extracted: 003954
Extracted: 003956
Extracted: 003963
Extracted: 003965
Extracted: 003966
Extracted: 003967
Extracted: 003969
Extracted: 003970
Extracted: 003971
Extracted: 003972
Extracted: 003974
Extracted: 003976
Extracted: 003977
Extracted: 003978
Extracted: 003985
Extracted: 003987
Extracted: 003988
Extracted: 003990
Extracted: 003992
Extracted: 003994
Extracted: 003996
Extracted: 003998
Extracted: 004001
Extracted: 004002
Extracted: 004008
Extracted: 004012
Extracted: 004013
Extracted: 004015
Extracted: 004016
Extracted: 004019
Extracted:

Extracted: 005172
Extracted: 005176
Extracted: 005179
Extracted: 005183
Extracted: 005185
Extracted: 005187
Extracted: 005192
Extracted: 005194
Extracted: 005195
Extracted: 005205
Extracted: 005206
Extracted: 005207
Extracted: 005208
Extracted: 005218
Extracted: 005219
Extracted: 005221
Extracted: 005224
Extracted: 005227
Extracted: 005228
Extracted: 005230
Extracted: 005231
Extracted: 005235
Extracted: 005236
Extracted: 005239
Extracted: 005240
Extracted: 005243
Extracted: 005249
Extracted: 005252
Extracted: 005254
Extracted: 005256
Extracted: 005258
Extracted: 005259
Extracted: 005263
Extracted: 005268
Extracted: 005271
Extracted: 005273
Extracted: 005278
Extracted: 005281
Extracted: 005282
Extracted: 005283
Extracted: 005289
Extracted: 005290
Extracted: 005292
Extracted: 005294
Extracted: 005298
Extracted: 005299
Extracted: 005304
Extracted: 005305
Extracted: 005307
Extracted: 005308
Extracted: 005309
Extracted: 005313
Extracted: 005315
Extracted: 005322
Extracted: 005325
Extracted:

Extracted: 006509
Extracted: 006510
Extracted: 006511
Extracted: 006512
Extracted: 006515
Extracted: 006516
Extracted: 006517
Extracted: 006518
Extracted: 006521
Extracted: 006522
Extracted: 006529
Extracted: 006531
Extracted: 006540
Extracted: 006541
Extracted: 006544
Extracted: 006546
Extracted: 006547
Extracted: 006550
Extracted: 006551
Extracted: 006552
Extracted: 006554
Extracted: 006556
Extracted: 006558
Extracted: 006560
Extracted: 006561
Extracted: 006563
Extracted: 006566
Extracted: 006569
Extracted: 006571
Extracted: 006572
Extracted: 006573
Extracted: 006576
Extracted: 006578
Extracted: 006579
Extracted: 006582
Extracted: 006585
Extracted: 006586
Extracted: 006592
Extracted: 006599
Extracted: 006601
Extracted: 006604
Extracted: 006608
Extracted: 006609
Extracted: 006614
Extracted: 006618
Extracted: 006623
Extracted: 006626
Extracted: 006631
Extracted: 006636
Extracted: 006641
Extracted: 006643
Extracted: 006644
Extracted: 006646
Extracted: 006647
Extracted: 006653
Extracted:

In [114]:
for f in set([file[:-4].lower() for file in os.listdir('../images/kitti')]):
    out = convertToXMLFormat(r'E:\Cloud\GIT\CVEET\images\kitti\{}.jpg'.format(f), convertFromKITTI)
    with open(r'E:\Cloud\GIT\CVEET\images\kitti\{}.xml'.format(f), 'w') as xml:
        xml.write('{}\n'.format(out))
        print('Processed', f)

Processed 003926
Processed 004651
Processed 007436
Processed 003480
Processed 002434
Processed 005698
Processed 005437
Processed 000283
Processed 000529
Processed 000430
Processed 000726
Processed 006566
Processed 006893
Processed 001060
Processed 003222
Processed 004278
Processed 005478
Processed 007017
Processed 002521
Processed 004088
Processed 006894
Processed 004428
Processed 003137
Processed 006914
Processed 000576
Processed 001210
Processed 001342
Processed 001448
Processed 001387
Processed 003201
Processed 004951
Processed 000589
Processed 006712
Processed 002516
Processed 003606
Processed 005079
Processed 004149
Processed 004618
Processed 001496
Processed 003332
Processed 005065
Processed 006713
Processed 000889
Processed 006411
Processed 002676
Processed 006297
Processed 007291
Processed 007398
Processed 000937
Processed 002114
Processed 004319
Processed 003158
Processed 002530
Processed 002045
Processed 005042
Processed 001822
Processed 007127
Processed 004315
Processed 0051

Processed 003187
Processed 006092
Processed 002287
Processed 000160
Processed 001235
Processed 002770
Processed 002630
Processed 006501
Processed 003761
Processed 006582
Processed 000964
Processed 005867
Processed 003667
Processed 004559
Processed 007271
Processed 004556
Processed 007113
Processed 000953
Processed 002123
Processed 005278
Processed 002920
Processed 003954
Processed 002765
Processed 001255
Processed 003264
Processed 005991
Processed 005325
Processed 001869
Processed 000363
Processed 000365
Processed 003837
Processed 004020
Processed 007240
Processed 004205
Processed 003166
Processed 004928
Processed 002240
Processed 000001
Processed 004450
Processed 002261
Processed 003366
Processed 000418
Processed 001252
Processed 001818
Processed 007437
Processed 000300
Processed 000465
Processed 007155
Processed 000231
Processed 005828
Processed 002602
Processed 005151
Processed 005322
Processed 002116
Processed 001737
Processed 002131
Processed 002507
Processed 006903
Processed 0010

Processed 004242
Processed 005683
Processed 003115
Processed 007080
Processed 004129
Processed 007304
Processed 001931
Processed 002828
Processed 000516
Processed 002009
Processed 001532
Processed 003387
Processed 007356
Processed 001621
Processed 005252
Processed 006643
Processed 006691
Processed 000078
Processed 002586
Processed 005294
Processed 005308
Processed 004361
Processed 003970
Processed 003292
Processed 006702
Processed 000054
Processed 007234
Processed 004949
Processed 005060
Processed 005954
Processed 005731
Processed 000059
Processed 007167
Processed 006001
Processed 000259
Processed 000381
Processed 005307
Processed 006217
Processed 006964
Processed 002578
Processed 006359
Processed 006227
Processed 000164
Processed 005597
Processed 004955
Processed 003540
Processed 001377
Processed 007006
Processed 003223
Processed 005696
Processed 000115
Processed 000268
Processed 003784
Processed 000360
Processed 002347
Processed 002581
Processed 007324
Processed 007217
Processed 0034

Processed 005298
Processed 002771
Processed 005876
Processed 002222
Processed 001700
Processed 003105
Processed 006898
Processed 005856
Processed 005880
Processed 006107
Processed 006988
Processed 002810
Processed 001571
Processed 006455
Processed 004030
Processed 001486
Processed 001594
Processed 003449
Processed 005910
Processed 006703
Processed 003607
Processed 005817
Processed 004485
Processed 005961
Processed 000253
Processed 001278
Processed 000543
Processed 000777
Processed 000769
Processed 000925
Processed 004773
Processed 006824
Processed 007111
Processed 003433
Processed 000302
Processed 001672
Processed 004875
Processed 003965
Processed 003127
Processed 003204
Processed 000288
Processed 000134
Processed 003453
Processed 005547
Processed 005543
Processed 005374
Processed 006656
Processed 003637
Processed 002064
Processed 005315
Processed 005550
Processed 007095
Processed 003963
Processed 005471
Processed 003793
Processed 001833
Processed 003892
Processed 003576
Processed 0056

Processed 003865
Processed 007413
Processed 001726
Processed 004244
Processed 002352
Processed 006697
Processed 002617
Processed 007312
Processed 000866
Processed 004342
Processed 007354
Processed 000459
Processed 007083
Processed 001864
Processed 006383
Processed 001671
Processed 004744
Processed 006223
Processed 000560
Processed 005626
Processed 003117
Processed 001134
Processed 002755
Processed 000941
Processed 003905
Processed 004973
Processed 000088
Processed 004865
Processed 000803
Processed 003825
Processed 006181
Processed 002125
Processed 001098
Processed 001739
Processed 001515
Processed 003073
Processed 003238
Processed 004016
Processed 004219
Processed 004056
Processed 000162
Processed 005902
Processed 004192
Processed 004765
Processed 007219
Processed 006767
Processed 003403
Processed 004142
Processed 001427
Processed 002735
Processed 005518
Processed 007269
Processed 004390
Processed 006209
Processed 001271
Processed 002162
Processed 007086
Processed 003232
Processed 0058

Processed 001528
Processed 003437
Processed 005165
Processed 004077
Processed 000341
Processed 004501
Processed 005740
Processed 007282
Processed 006579
Processed 007466
Processed 004240
Processed 001805
Processed 006510
Processed 003259
Processed 001694
Processed 006899
Processed 005192
Processed 000881
Processed 006790
Processed 002381
Processed 004820
Processed 004374
Processed 006436
Processed 001555
Processed 001526
Processed 002065
Processed 002849
Processed 001973
Processed 006279
Processed 006680
Processed 000015
Processed 001534
Processed 004446
Processed 004998
Processed 005143
Processed 000289
Processed 001678
Processed 006930
Processed 005389
Processed 003590
Processed 002159
Processed 003466
Processed 002887
Processed 004153
Processed 005670
Processed 006083
Processed 003856
Processed 002189
Processed 003303
Processed 004521
Processed 001601
Processed 002390
Processed 001478
Processed 004451
Processed 007009
Processed 007415
Processed 002191
Processed 002938
Processed 0060

Misc converter...

In [5]:
for f in [f for f in os.listdir('../images/dark') if f.lower().endswith('.jpg')]:    
    img = cv2.imread('../images/dark/{}'.format(f), cv2.IMREAD_COLOR)
    cv2.imwrite('../images/dark/{}'.format(f), img, [int(cv2.IMWRITE_JPEG_QUALITY), 90])
    print('Handled', f)

Handled 2015_01879.jpg
Handled 2015_01880.jpg
Handled 2015_01881.jpg
Handled 2015_01882.jpg
Handled 2015_01883.jpg
Handled 2015_01884.jpg
Handled 2015_01885.jpg
Handled 2015_01886.jpg
Handled 2015_01887.jpg
Handled 2015_01888.jpg
Handled 2015_01889.jpg
Handled 2015_01890.jpg
Handled 2015_01891.jpg
Handled 2015_01892.jpg
Handled 2015_01893.jpg
Handled 2015_01894.jpg
Handled 2015_01895.jpg
Handled 2015_01896.jpg
Handled 2015_01897.jpg
Handled 2015_01898.jpg
Handled 2015_01899.jpg
Handled 2015_01900.jpg
Handled 2015_01901.jpg
Handled 2015_01902.jpg
Handled 2015_01903.jpg
Handled 2015_01904.jpg
Handled 2015_01905.jpg
Handled 2015_01906.jpg
Handled 2015_01907.jpg
Handled 2015_01908.jpg
Handled 2015_01909.jpg
Handled 2015_01910.jpg
Handled 2015_01911.jpg
Handled 2015_01912.jpg
Handled 2015_01913.jpg
Handled 2015_01914.jpg
Handled 2015_01915.jpg
Handled 2015_01916.jpg
Handled 2015_01917.jpg
Handled 2015_01918.jpg
Handled 2015_01919.jpg
Handled 2015_01920.jpg
Handled 2015_01921.jpg
Handled 201

Handled 2015_02244.jpg
Handled 2015_02245.jpg
Handled 2015_02246.jpg
Handled 2015_02247.jpg
Handled 2015_02248.jpg
Handled 2015_02249.jpg
Handled 2015_02250.jpg
Handled 2015_02251.jpg
Handled 2015_02252.jpg
Handled 2015_02253.jpg
Handled 2015_02254.jpg
Handled 2015_02255.jpg
Handled 2015_02256.jpg
Handled 2015_02257.jpg
Handled 2015_02258.jpg
Handled 2015_02259.jpg
Handled 2015_02260.jpg
Handled 2015_02261.jpg
Handled 2015_02262.jpg
Handled 2015_02263.jpg
Handled 2015_02264.jpg
Handled 2015_02265.jpg
Handled 2015_02266.jpg
Handled 2015_02267.jpg
Handled 2015_02268.jpg
Handled 2015_02269.jpg
Handled 2015_02270.jpg
Handled 2015_02271.jpg
Handled 2015_02272.jpg
Handled 2015_02273.jpg
Handled 2015_02274.jpg
Handled 2015_02275.jpg
Handled 2015_02276.jpg
Handled 2015_02277.jpg
Handled 2015_02278.jpg
Handled 2015_02279.jpg
Handled 2015_02280.jpg
Handled 2015_02281.jpg
Handled 2015_02282.jpg
Handled 2015_02283.jpg
Handled 2015_02284.jpg
Handled 2015_02285.jpg
Handled 2015_02286.jpg
Handled 201

Handled 2015_02602.jpg
Handled 2015_02603.jpg
Handled 2015_02604.jpg
Handled 2015_02605.jpg
Handled 2015_02606.jpg
Handled 2015_02607.jpg
Handled 2015_02608.jpg
Handled 2015_02609.jpg
Handled 2015_02610.jpg
Handled 2015_02611.jpg
Handled 2015_02612.jpg
Handled 2015_02613.jpg
Handled 2015_02614.jpg
Handled 2015_02615.jpg
Handled 2015_02616.jpg
Handled 2015_02617.jpg
Handled 2015_02618.jpg
Handled 2015_02619.jpg
Handled 2015_02620.jpg
Handled 2015_02621.jpg
Handled 2015_02622.jpg
Handled 2015_02623.jpg
Handled 2015_02624.jpg
Handled 2015_02625.jpg
Handled 2015_02626.jpg
Handled 2015_02627.jpg
Handled 2015_02628.jpg
Handled 2015_02629.jpg
Handled 2015_02630.jpg
Handled 2015_02631.jpg
Handled 2015_02632.jpg
Handled 2015_02633.jpg
Handled 2015_02634.jpg
Handled 2015_02635.jpg
Handled 2015_02636.jpg
Handled 2015_02637.jpg
Handled 2015_02638.jpg
Handled 2015_02639.jpg
Handled 2015_02640.jpg
Handled 2015_02641.jpg
Handled 2015_02642.jpg
Handled 2015_02643.jpg
Handled 2015_02644.jpg
Handled 201

Handled 2015_02963.jpg
Handled 2015_02964.jpg
Handled 2015_02965.jpg
Handled 2015_02966.jpg
Handled 2015_02967.jpg
Handled 2015_02968.jpg
Handled 2015_02969.jpg
Handled 2015_02970.jpg
Handled 2015_02971.jpg
Handled 2015_02972.jpg
Handled 2015_02973.jpg
Handled 2015_02974.jpg
Handled 2015_02975.jpg
Handled 2015_02976.jpg
Handled 2015_02977.jpg
Handled 2015_02978.jpg
Handled 2015_02979.jpg
Handled 2015_02980.jpg
Handled 2015_02981.jpg
Handled 2015_02982.jpg
Handled 2015_02983.jpg
Handled 2015_02984.jpg
Handled 2015_02985.jpg
Handled 2015_02986.jpg
Handled 2015_02987.jpg
Handled 2015_02988.jpg
Handled 2015_02989.jpg
Handled 2015_02990.jpg
Handled 2015_02991.jpg
Handled 2015_02992.jpg
Handled 2015_02993.jpg
Handled 2015_02994.jpg
Handled 2015_02995.jpg
Handled 2015_02996.jpg
Handled 2015_02997.jpg
Handled 2015_02998.jpg
Handled 2015_02999.jpg
Handled 2015_03000.jpg
Handled 2015_03001.jpg
Handled 2015_03002.jpg
Handled 2015_03003.jpg
Handled 2015_03004.jpg
Handled 2015_03005.jpg
Handled 201

Handled 2015_06026.jpg
Handled 2015_06027.jpg
Handled 2015_06028.jpg
Handled 2015_06029.jpg
Handled 2015_06030.jpg
Handled 2015_06031.jpg
Handled 2015_06032.jpg
Handled 2015_06033.jpg
Handled 2015_06034.jpg
Handled 2015_06035.jpg
Handled 2015_06036.jpg
Handled 2015_06037.jpg
Handled 2015_06038.jpg
Handled 2015_06039.jpg
Handled 2015_06040.jpg
Handled 2015_06041.jpg
Handled 2015_06042.jpg
Handled 2015_06043.jpg
Handled 2015_06044.jpg
Handled 2015_06045.jpg
Handled 2015_06046.jpg
Handled 2015_06047.jpg
Handled 2015_06048.jpg
Handled 2015_06049.jpg
Handled 2015_06050.jpg
Handled 2015_06051.jpg
Handled 2015_06052.jpg
Handled 2015_06053.jpg
Handled 2015_06054.jpg
Handled 2015_06055.jpg
Handled 2015_06056.jpg
Handled 2015_06057.jpg
Handled 2015_06058.jpg
Handled 2015_06059.jpg
Handled 2015_06060.jpg
Handled 2015_06061.jpg
Handled 2015_06062.jpg
Handled 2015_06063.jpg
Handled 2015_06064.jpg
Handled 2015_06065.jpg
Handled 2015_06066.jpg
Handled 2015_06067.jpg
Handled 2015_06068.jpg
Handled 201

Handled 2015_06396.jpg
Handled 2015_06397.jpg
Handled 2015_06398.jpg
Handled 2015_06399.jpg
Handled 2015_06400.jpg
Handled 2015_06401.jpg
Handled 2015_06402.jpg
Handled 2015_06403.jpg
Handled 2015_06404.jpg
Handled 2015_06405.jpg
Handled 2015_06406.jpg
Handled 2015_06407.jpg
Handled 2015_06408.jpg
Handled 2015_06409.jpg
Handled 2015_06410.jpg
Handled 2015_06411.jpg
Handled 2015_06412.jpg
Handled 2015_06413.jpg
Handled 2015_06414.jpg
Handled 2015_06415.jpg
Handled 2015_06416.jpg
Handled 2015_06417.jpg
Handled 2015_06418.jpg
Handled 2015_06419.jpg
Handled 2015_06420.jpg
Handled 2015_06421.jpg
Handled 2015_06422.jpg
Handled 2015_06423.jpg
Handled 2015_06424.jpg
Handled 2015_06425.jpg
Handled 2015_06426.jpg
Handled 2015_06427.jpg
Handled 2015_06428.jpg
Handled 2015_06429.jpg
Handled 2015_06430.jpg
Handled 2015_06431.jpg
Handled 2015_06432.jpg
Handled 2015_06433.jpg
Handled 2015_06434.jpg
Handled 2015_06435.jpg
Handled 2015_06436.jpg
Handled 2015_06437.jpg
Handled 2015_06438.jpg
Handled 201

Handled 2015_06756.jpg
Handled 2015_06757.jpg
Handled 2015_06758.jpg
Handled 2015_06759.jpg
Handled 2015_06760.jpg
Handled 2015_06761.jpg
Handled 2015_06762.jpg
Handled 2015_06763.jpg
Handled 2015_06764.jpg
Handled 2015_06765.jpg
Handled 2015_06766.jpg
Handled 2015_06767.jpg
Handled 2015_06768.jpg
Handled 2015_06769.jpg
Handled 2015_06770.jpg
Handled 2015_06771.jpg
Handled 2015_06772.jpg
Handled 2015_06773.jpg
Handled 2015_06774.jpg
Handled 2015_06775.jpg
Handled 2015_06776.jpg
Handled 2015_06777.jpg
Handled 2015_06778.jpg
Handled 2015_06779.jpg
Handled 2015_06780.jpg
Handled 2015_06781.jpg
Handled 2015_06782.jpg
Handled 2015_06783.jpg
Handled 2015_06784.jpg
Handled 2015_06785.jpg
Handled 2015_06786.jpg
Handled 2015_06787.jpg
Handled 2015_06788.jpg
Handled 2015_06789.jpg
Handled 2015_06790.jpg
Handled 2015_06791.jpg
Handled 2015_06792.jpg
Handled 2015_06793.jpg
Handled 2015_06794.jpg
Handled 2015_06795.jpg
Handled 2015_06796.jpg
Handled 2015_06797.jpg
Handled 2015_06798.jpg
Handled 201